In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle


In [ ]:
#저장 파일명 지정하기
model_path = "fashion_mnist_model.keras" #확장자를 keras로 하거나 없거나
history_path = "fashion_mnist_history.bin" #이 파일 확장자는 마음대로

In [ ]:
def load_data():
  #이 데이터는 케라스가 제공한다 -> 유명함
  (X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

  print(X_train.shape)
  print(y_train.shape)

  print(X_test.shape)
  print(y_test.shape)

  #60000, 28, 28   흑백이미지  28 by 28  훈련셋이 60000, 테스트셋이 10000
  print(y_test[:10])
  return X_train, y_train, X_test, y_test

In [ ]:
#1. 결측치나 이상치가 없다. 이유는 이미지파일 불러서 numpy로 바꾼거라
#2. 표준화나 정규화
#3. target 의 라벨인코딩 또는 원핫인코딩  하기싫으면  loss에 sparse_categorical_crossentropy를 사용하면 된다
def preprocessing(X_train, y_train, X_test, y_test):
  X_train = X_train.astype("float32")/255
  X_test = X_test.astype("float32")/255
  X_train = np.expand_dims(X_train, -1) #차원버전
  X_test = np.expand_dims(X_test, -1) #차원추가
  print(X_train.shape)
  print(X_test.shape)
  return X_train,  y_train, X_test, y_test




In [ ]:
def getModel():
  data_argumentation = keras.Sequential(
      [
          layers.RandomFlip("horizontal"),
          layers.RandomRotation(0.1),
          layers.RandomZoom(0.2)
      ]
  )

  model = keras.Sequential( [
      #data_argumentation,,
      layers.Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=((28,28,1))),
      layers.MaxPooling2D(pool_size=(2,2)),
      layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
      layers.MaxPooling2D(pool_size=(2,2)),
      layers.Flatten(),  #CNN와 완전연결망을 연결시킨다
      layers.Dense(64, activation='relu'),
      layers.Dense(32, activation='relu'),
      layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  model.summary()
  return model




In [ ]:
def study(model, X_train, y_train, X_test, y_test):

  early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
  history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping])

  model.save(model_path)
  with open(history_path, 'wb') as f:
    pickle.dump(history.history, f)

  return history


In [ ]:
X_train, y_train, X_test, y_test = load_data()
X_train, y_train, X_test, y_test = preprocessing(X_train,  y_train, X_test, y_test)
model = getModel()
study(model, X_train, y_train, X_test, y_test)






(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
[9 2 1 1 6 1 4 6 5 7]
(60000, 28, 28, 1)
(10000, 28, 28, 1)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_26 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,690 (483.16 KB)

 Trainable params: 123,690 (483.16 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 43s 27ms/step - accuracy: 0.7499 - loss: 0.7017 - val_accuracy: 0.8744 - val_loss: 0.3579
Epoch 2/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.8742 - loss: 0.3461 - val_accuracy: 0.8880 - val_loss: 0.3090
Epoch 3/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 29ms/step - accuracy: 0.8950 - loss: 0.2889 - val_accuracy: 0.8885 - val_loss: 0.3096
Epoch 4/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.9068 - loss: 0.2549 - val_accuracy: 0.9051 - val_loss: 0.2649
Epoch 5/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 43s 29ms/step - accuracy: 0.9201 - loss: 0.2210 - val_accuracy: 0.8988 - val_loss: 0.2776
Epoch 6/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 80s 27ms/step - accuracy: 0.9254 - loss: 0.2024 - val_accuracy: 0.9009 - val_loss: 0.2814
Epoch 7/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 28ms/step - accuracy: 0.9319 - loss: 0.1836 - val_accuracy: 0.9107 - val_loss: 0.2571
Epoch 8/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 80s 27ms/step - accuracy: 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
#평가하기
def evalute_model(X_test, y_test):
  model = keras.models.load_model(model_path)
  with open(history_path, 'rb') as f:
    history = pickle.load(f)
  #모델하고 히스토리 불러오기

  #예측하기
  y_pred = model.predict(X_test) #예측확률
  print(y_pred.shape)  #softmax함수가 하는일이 실제 출력값들은 가중치
  #가중치들을 확률로 바꿔서 전달하는 함수가 softmax
  print(y_pred[:10])
  y_pred = np.argmax(y_pred, axis=1) #최대값 위치를 찾아온다 벡터연산
  print(y_pred[:10])
  print(y_test[:10])

  #모델 평가
  loss, accuracy = model.evaluate(X_test, y_test)
  print(f"테스트셋 손실 {loss}")
  print(f"테스트셋 정확도 {accuracy}")

  #머신러닝에서 썼던 평가방법을 여기도 쓸 수 있다 - 혼동행렬
  print("혼동행렬")
  print(confusion_matrix(y_test, y_pred))

  print("\n\n\n분류보고서")
  class_names = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt',
                 'Sneaker', 'Bag', 'Ankle boot']

  print(classification_report(y_test, y_pred, target_names=class_names))








evalute_model(X_test, y_test)



313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
(10000, 10)
[[9.96526126e-11 2.18388103e-11 1.43640823e-13 2.52989771e-08
  2.18241633e-10 3.56191210e-09 1.17473287e-09 7.22525192e-06
  2.74910194e-12 9.99992669e-01]
 [7.89188270e-10 1.32248509e-11 9.99123096e-01 1.91376148e-06
  2.50493059e-07 1.38791329e-14 8.74724530e-04 7.66645370e-22
  1.87991911e-09 9.02932988e-14]
 [1.00685053e-17 9.99999940e-01 2.45215128e-21 2.24464054e-20
  6.81161398e-29 2.95376458e-22 3.46656854e-20 6.47918380e-31
  1.56192726e-27 1.87288393e-30]
 [4.12441921e-19 9.99999940e-01 6.48837112e-20 2.07661147e-19
  5.26988359e-26 5.84701700e-27 2.38827852e-19 1.80669919e-33
  1.04440360e-29 3.82887380e-31]
 [5.08368015e-01 3.38573995e-11 1.04255640e-04 2.29169743e-07
  2.39833817e-03 9.67311578e-11 4.89129186e-01 1.10370285e-12
  1.16786199e-11 2.93070207e-10]
 [4.74599820e-20 9.99999940e-01 7.72738493e-20 1.32935460e-17
  1.80937303e-28 3.84001869e-24 6.37791731e-20 7.65226389e-34
  9.71132269e-30 1.52708142e-31]
 [6.